In [3]:
import util_gui
from PIL import Image
import glob
import os
import time
import cv2
import numpy as np
import pandas as pd
import pyautogui as gui

In [4]:
#Identificador de Fleet Power PVP Azur Lane

#Imagem base para a analise e retirada dos numeros
#(trocar a imagem por uma tirada de screenshot da tela de pvp)
img_base_pvp= cv2.imread("C:\\Users\\EricR\\Desktop\\exemplo_pvp.png")
img_copy = img_base_pvp.copy()
img_copy = img_copy[240:339,148:1064,:]
numero_players = 4 #Numero de players a serem analisados
fleet_power = np.empty((4,2),dtype = int)
#Threshold para identificacao dos numeros_basicos
#Numeros_basicos correspondem aos numeros na folder "Imagens\Numbers"
threshold = 0.7
fleet_power_max = 15500
#Threshold para identificacao dos "Gen"
#Gen corresponde ao identificador de cada player no pvp
#A identificação é feita onde cada player tem sua Fleet Power antecedida pela frase "General Strength"
threshold_gen = 0.8

template_gen = cv2.imread(os.path.join(os.getcwd(),r"Images\PVP\gen.png")) #Template do gen
h_gen,w_gen = template_gen.shape[:2] #h_gen e w_gen sao usados para pintar a utilizada de area de vermelho (cobrir os gens já usados)
prev_max_val_gen, prev_max_loc_gen = None, None #evitar repeticao 


template_numeros = [] #armazena os templates dos numeros_basicos
h_numeros = [] #armazena as alturas dos numeros_basicos
w_numeros = [] #armazena as widths dos numeros_basicos

#lista com path de todos os numeros_basicos
lista_path_numeros = glob.glob(os.path.join(os.getcwd(),r"Images\PVP\Numeros\*.png")) 

#armazena qual numero_basico corresponde a tal index (pois ha repeticao de numeros)
#Como existem varios numeros por numero_basico, a variavel "array_contagem_numeros" faz a seguinte funcao
#index da max_val_numeros_array pela funcao index = np.argmax(max_val_numeros_array)
#array_contagem_numeros[index] gera o numero_basico correspondente 
array_contagem_numeros = np.empty(len(lista_path_numeros),dtype = int)

#Iterador_1
#Popula as variaveis de template_numeros,h_numeros e w_numeros que sao informacoes dos numeros_basicos
for iter_template_int, iter_template_path in enumerate(lista_path_numeros):
    
    #Como o iter_template_path contem o numero na forma ...\i_j.png
    #i = numero_basico
    #j = versao do numero_basico
    #Extrai-se o "i" no slicing [-7:-6]
    array_contagem_numeros[iter_template_int] = iter_template_path[-7:-6] 
    
    #Populacao das variaveis dos numeros_basicos
    template_numeros.append(cv2.imread(iter_template_path))
    h,w = template_numeros[iter_template_int].shape[:2]
    h_numeros.append(h)
    w_numeros.append(w)

#Armazena as variaveis max_..._gen no Iterador_1
#Utilizadas no Iterador_2
max_loc_gen_array = []

#Iterador_2
#Popula as variaveis max_..._gen_array
for iter_gen in range(numero_players):

    #Gera o match dessa iteracao
    res = cv2.matchTemplate(img_copy, template_gen, cv2.TM_CCOEFF_NORMED)
    
    #Pega a regiao com melhor match pela funcao cv2.minMaxLoc
    #Codigo base de:
    #https://stackoverflow.com/questions/50579050/template-matching-with-multiple-objects-in-opencv-python
    _,max_val_gen,_,max_loc_gen = cv2.minMaxLoc(res)
    
    #Prevenir que haja sobrescricao de "gen"s no mesmo local
    if((prev_max_val_gen == max_val_gen) and (prev_max_loc_gen == max_loc_gen)):
        print("Repeating values \(gen case\)")
        break
    prev_max_val_gen, prev_max_loc_gen = max_val_gen, max_loc_gen
    
    #Popula as variaveis max_..._gen_array 
    max_loc_gen_array.append(max_loc_gen)
    
    #Se nao houver mais gens, break
    if(max_val_gen < threshold_gen):
        print(f"No more matching values. Max_val_gen = {max_val_gen}")
        break
        
    #Colorindo a parte do gen no img_copy
    #A regiao pintada nao consegue ser mais utilizada nos matches -> evita sobreposicao
    img_copy[max_loc_gen[1]:max_loc_gen[1]+h_gen:, max_loc_gen[0]:max_loc_gen[0]+w_gen, 0] = 0
    img_copy[max_loc_gen[1]:max_loc_gen[1]+h_gen:, max_loc_gen[0]:max_loc_gen[0]+w_gen, 1] = 0
    img_copy[max_loc_gen[1]:max_loc_gen[1]+h_gen:, max_loc_gen[0]:max_loc_gen[0]+w_gen, 2] = 255

#Sorta a variavel do local das gens
#Pega do mais a esquerda (player_1) ate o mais a direita (player_4)
max_loc_gen_array = sorted(max_loc_gen_array,key = lambda x:x[0])

#Iterador_3
#Gera as identificacoes dos numeros_basicos dentro de uma regiao(img_copy_gen) da img_copy

for iter_gen in range(numero_players):
    #A copia img_copy sera de onde retiraramos as regioes(img_copy_gen recortada)
    #A copia img_copy_gen sera de onde retiraremos os numeros
    img_copy_gen = img_copy.copy()
    
    #Extraindo a regiao da imagem para analise no img_copy_gen
    img_copy_gen = img_copy_gen[max_loc_gen_array[iter_gen][1]:max_loc_gen_array[iter_gen][1]+70,max_loc_gen_array[iter_gen][0]:max_loc_gen_array[iter_gen][0]+90,:]
    
    #Iterador_3.1
    #Verifica se ha algum numero; Se nao houver, para o proximo i
    #Se houver, verifica qual numero que mais se assemelha a ele, dentro do iterador_3.1.1
    #Ex: 
    #   Com o "i", ha algum numero que pareca (dentro do while_1_iterador_3.1)
    #   Se sim, procurar outros numeros para ver o que mais parece, dentro do iterador_3.1.1
    #   Se nao, para o "i+1"
    array_numeros_identificados = []
    for i in range(len(array_contagem_numeros)):
        prev_max_val, prev_max_loc = None, None #Previne sobreposicao de matches (utilizado anteriormente no iterador_1)
        max_val = 1
        
        #While_1_iterador_3.1
        while(max_val > threshold):
            res = cv2.matchTemplate(img_copy_gen, template_numeros[i], cv2.TM_CCOEFF_NORMED)
            _,max_val,_,max_loc = cv2.minMaxLoc(res)

            #Previne sobreposicao de matches
            if((prev_max_val == max_val) and (prev_max_loc == max_loc)): 
                #print(f"Repeating values in {i}")
                break
            prev_max_val, prev_max_loc =  max_val, max_loc
            
            #Se houver algum numero parecido com "i"
            if(max_val > threshold):
                #print(f"Numero parecido com {i}")
                
                #Armazenam os max_val e max_loc dos numeros_basicos
                #max_val_numeros_array: vai identificar qual numero_basico mais se parece com o numero parecido com "i"
                #max_loc_numeros_array: armazena as coordenadas dos numeros_basicos identificador
                #Obs: max_loc_numeros armazena todos, ate os que nao se parecam com "i")
                max_val_numeros_array = np.zeros(len(array_contagem_numeros))
                max_loc_numeros_array = np.zeros(len(array_contagem_numeros),dtype = tuple)
                
                #Iterador_3.1.1
                #Extrai o max_val_numeros_array com o numero mais parecido
                for j in range(len(array_contagem_numeros)):
                    res = cv2.matchTemplate(img_copy_gen, template_numeros[j], cv2.TM_CCOEFF_NORMED)
                    _,max_val_numeros_array[j],_,max_loc_numeros_array[j] = cv2.minMaxLoc(res)
                
                #Identifica qual a index do numero_basico mais parecido com "i"/Numero identificado
                index = np.argmax(max_val_numeros_array)
                #print(f"\t\tMais parecido com {index}")
                #Conta qual numero_basico que foi o mais parecido
                #array_numeros[array_contagem_numeros[index]] += 1
                
                #Extrai variaveis h,w de max_loc_numeros_array do numero_basico mais parecido para preencher o retangulo
                h_max_loc = max_loc_numeros_array[index][1]
                w_max_loc = max_loc_numeros_array[index][0]
                
                #Preenche o retangulo com a h,w do numero_basico mais parecido
                img_copy_gen[h_max_loc:h_max_loc+h_numeros[index], w_max_loc:w_max_loc+w_numeros[index], 0] = 0
                img_copy_gen[h_max_loc:h_max_loc+h_numeros[index], w_max_loc:w_max_loc+w_numeros[index], 1] = 0
                img_copy_gen[h_max_loc:h_max_loc+h_numeros[index], w_max_loc:w_max_loc+w_numeros[index], 2] = 255
                
                #Escreve qual foi o numero_basico mais parecido em cima da regiao pintada
                cv2.putText(img_copy_gen,f"{array_contagem_numeros[index]}",(w_max_loc,h_max_loc),1,1,(255,255,255))
                #max_val = np.max(max_val_numeros_array) (acho que nao precisa dessa linha)
                
                array_numeros_identificados.append((array_contagem_numeros[index],max_loc_numeros_array[index]))
        #print(f"Finished iterating for number {i}\n\n")
        
    print(f"Finished image {iter_gen+1}")
    
    #Identificacao de numero
    for iter_write_numero in range(2):
        #Sort de acordo com a posicao y dos numeros
        array_numeros_identificados = sorted(array_numeros_identificados,key=lambda x:x[1][1])
        
        #Retirando numeros com altura parecida
        copy_array_numeros_identificados = [x for x in array_numeros_identificados if ((x[1][1]-array_numeros_identificados[0][1][1]) < 10)]
        
        #Sort de acordo com a posicao x dos numeros
        copy_array_numeros_identificados = sorted(copy_array_numeros_identificados,key=lambda x:x[1][0])
        
        copy_array_numeros_identificados = ''.join([str(x[0]) for x in copy_array_numeros_identificados])
        array_numeros_identificados = array_numeros_identificados[len(copy_array_numeros_identificados):]
        fleet_power[iter_gen][iter_write_numero] = int(copy_array_numeros_identificados)
        print(copy_array_numeros_identificados)
    cv2.imwrite("teste.png",img_copy_gen)
    result = cv2.imread("teste.png")
    cv2.imshow("teste",result)
    cv2.waitKey(0)

#Decisao de qual player jogar o pvp, dado o fleet_power
#Decidindo pela menor soma de fleet_power(ataque e defesa)
fleet_power_conjunto = [(x[0]+x[1])/2 for x in fleet_power]
if np.min(fleet_power_conjunto) > fleet_power_max:
    #Decidir o que fazer quando o fleet_power for maior do que 30K
    #Provavelmente resetar?
    pass
player_escolhido = np.argmin(fleet_power_conjunto)
x_player_escolhido,y_player_escolhido = max_loc_gen_array[player_escolhido]
gui.click(x_player_escolhido,y_player_escolhido)

cv2.destroyAllWindows()

Finished image 1
9767
12475
Finished image 2
15299
14665
Finished image 3
15437
15767
Finished image 4
1214
11869


NameError: name 'fleet_power_max' is not defined